In [1]:
!pip install transformers
!pip install sentence-transformers
!pip install qdrant-client
!pip install PyMuPDF  # for parsing PDFs
!pip install pypdf2   # Alternative for PDF parsing if needed
!pip install langchain


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 249.1/249.1 kB 11.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 258.9/258.9 kB 7.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.4/2.4 MB 24.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.7/5.7 MB 40.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.4/76.4 kB 1.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.9/77.9 kB 3.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 57.5/57.5 kB 3.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 316.6/316.6 kB 21.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 3.1 MB/s eta 0:00:00
  Attempting uninstall: protobuf
    Found existing installation: protobuf 3.20.3
    Uninstalling protobuf-3.20.3:
      Successfully uninstalled protobuf-3.20.3
  Attempting uninstall: grpcio
    Found existing installation: grpcio 1.64.1
    Uninstalling grpcio-1.64.1:
      Successfully 

In [2]:
!pip install -U langchain-community


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.3/2.3 MB 43.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.3/49.3 kB 4.6 MB/s eta 0:00:00


In [3]:
!pip install  gradio

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 18.1/18.1 MB 37.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 318.7/318.7 kB 20.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 94.0/94.0 kB 6.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.3/10.3 MB 45.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.8/62.8 kB 5.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 71.4/71.4 kB 4.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 130.2/130.2 kB 7.9 MB/s eta 0:00:00
  Attempting uninstall: tomlkit
    Found existing installation: tomlkit 0.13.2
    Uninstalling tomlkit-0.13.2:
      Successfully uninstalled tomlkit-0.13.2


In [5]:
import torch
from transformers import pipeline, AutoModelForQuestionAnswering, AutoTokenizer
from sentence_transformers import SentenceTransformer
from qdrant_client import QdrantClient
from qdrant_client.http.models import VectorParams, Distance, PointStruct
from uuid import uuid4

# Initialize Qdrant client (assumes a local Qdrant instance running)
client = QdrantClient(":memory:")  # Use ":memory:" for in-memory storage

# Create the collection with vector parameters
client.recreate_collection(
    collection_name="my_documents",
    vectors_config=VectorParams(size=384, distance=Distance.COSINE)
)

# Load the pre-trained sentence transformer model for multilingual embeddings
embedding_model = SentenceTransformer('sentence-transformers/paraphrase-multilingual-MiniLM-L12-v2')

# Load the pre-trained multilingual QA model from Hugging Face
qa_model_name = "deepset/xlm-roberta-base-squad2"
qa_model = AutoModelForQuestionAnswering.from_pretrained(qa_model_name)
qa_tokenizer = AutoTokenizer.from_pretrained(qa_model_name)
device = 0 if torch.cuda.is_available() else -1
qa_pipeline = pipeline("question-answering", model=qa_model, tokenizer=qa_tokenizer, device=device)

# Function to convert text into documents with pagination
def text_to_docs(text, chars_per_page=2000):
    pages = [text[i:i+chars_per_page] for i in range(0, len(text), chars_per_page)]
    docs = [{"id": str(uuid4()), "page_content": page} for page in pages]
    return docs

# Function to handle user input
def handle_text(text):
    pages = text_to_docs(text)

    # Create embeddings and store them in Qdrant
    client.recreate_collection(
        collection_name="my_documents",
        vectors_config=VectorParams(size=384, distance=Distance.COSINE)
    )

    for page in pages:
        embedding = embedding_model.encode(page['page_content'])
        point = PointStruct(
            id=page['id'],
            vector=embedding.tolist(),
            payload={"page_content": page['page_content']}
        )
        client.upsert(collection_name="my_documents", points=[point])

    return "Text processed and ready for questioning."

# Function to handle user questions
def ask_question(question, text):
    try:
        # Process and store text
        handle_text(text)

        # Generate the embedding for the question
        question_embedding = embedding_model.encode(question).tolist()

        # Search for the most similar document in the collection
        search_result = client.search(
            collection_name="my_documents",
            query_vector=question_embedding,
            limit=1  # Retrieve the top result
        )

        if search_result:
            context = search_result[0].payload['page_content']

            # Use the QA pipeline to answer the question based on the context
            qa_input = {"question": question, "context": context}
            result = qa_pipeline(qa_input)
            return result['answer']
        else:
            return "Sorry, I couldn't find an answer to that question."
    except Exception as e:
        return f"An error occurred: {str(e)}"

# Interactive chat loop
def chat(text, question):
    return ask_question(question, text)

# Run the Gradio interface
import gradio as gr

interface = gr.Interface(
    fn=chat,
    inputs=[
        gr.Textbox(lines=10, placeholder="Enter your text here...", label="Enter Text"),
        gr.Textbox(lines=1, placeholder="Ask a question...", label="Question")
    ],
    outputs="text",
    title="Text-Based Question Answering Bot",
    description="Enter any text paragraph and ask questions about it."
)

interface.launch()


<ipython-input-5-bce796e4b6b6>:12: DeprecationWarning: `recreate_collection` method is deprecated and will be removed in the future. Use `collection_exists` to check collection existence and `create_collection` instead.
  client.recreate_collection(


sentencepiece.bpe.model:   0%|          | 0.00/5.07M [00:00<?, ?B/s]

Setting queue=True in a Colab notebook requires sharing enabled. Setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
Running on public URL: https://6320adb280f8455524.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from Terminal to deploy to Spaces (https://huggingface.co/spaces)


In [6]:
!pip install openai

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 374.2/374.2 kB 14.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 318.9/318.9 kB 22.9 MB/s eta 0:00:00


In [7]:
!pip install gradio

In [8]:
import gradio as gr
import openai

# Set your OpenAI API key here
openai.api_key = 'sk-41qQFnRM-0d-uUGy0akjHKCnYfBZr1Txx27DvgZg3qT3BlbkFJZTCDfsH4Tyn1x5E-zRtl0LH1gIUdH2pyf-TEOhitMA'

def get_answer(question):
    try:
        # Query the OpenAI API using the updated interface
        response = openai.ChatCompletion.create(
            model="gpt-3.5",  # or "gpt-4" if you prefer
            messages=[
                {"role": "user", "content": question}
            ]
        )
        answer = response.choices[0].message['content'].strip()
        return answer
    except Exception as e:
        return f"An error occurred: {str(e)}"

# Define the Gradio interface
def chat(question):
    return get_answer(question)

interface = gr.Interface(
    fn=chat,
    inputs=gr.Textbox(lines=2, placeholder="Ask any question here...", label="Question"),
    outputs="text",
    title="General Knowledge Chatbot",
    description="Ask questions on a wide range of topics and get detailed answers."
)

interface.launch()


Setting queue=True in a Colab notebook requires sharing enabled. Setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
Running on public URL: https://57de855f17e4697dae.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from Terminal to deploy to Spaces (https://huggingface.co/spaces)


In [9]:
import torch
print(torch.cuda.is_available())  # Should return True if GPU is active


True


In [10]:
from transformers import GPTNeoForCausalLM  # Correct model for GPT-Neo


In [13]:
import gradio as gr
from transformers import pipeline

# Load a language model from Hugging Face
generator = pipeline("text-generation", model="EleutherAI/gpt-neo-2.7B")

def generate_response(prompt):
    try:
        # Generate a response using the GPT-Neo model
        response = generator(prompt, max_length=150, do_sample=True, temperature=0.7, top_p=0.9)
        return response[0]["generated_text"].strip()
    except Exception as e:
        return f"An error occurred: {str(e)}"

# Define the Gradio interface
interface = gr.Interface(
    fn=generate_response,
    inputs=gr.Textbox(lines=2, placeholder="Ask any question here...", label="Question"),
    outputs="text",
    title="GPT-Neo Chatbot",
    description="Ask questions and get detailed responses from GPT-Neo."
)

# Launch the Gradio interface with public sharing enabled
interface.launch(share=True)


/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(
Hardware accelerator e.g. GPU is available in the environment, but no `device` argument is passed to the `Pipeline` object. Model will be on CPU.


Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
Running on public URL: https://0b6bb8886791880444.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from Terminal to deploy to Spaces (https://huggingface.co/spaces)
